## Selección de jugadores

Seleccionaremos algunos jugadores, aquellos con más partidas y algunos conocidos para ilustrar nuestro estudio posterior. Las partidas que consideraremos serán las de estos jugadores a lo largo de su trayectoria profesional (últimos X años) junto con la de todos los jugadores en los últimos (x meses / años). Así, tendremos suficientes partidas para realizar algunos estudios posteriores (Bot).

In [1]:
import chess.pgn
import os
from collections import defaultdict
from tqdm import tqdm  # Barra de progreso
from pyspark.sql import SparkSession
from io import StringIO
import subprocess

### Buscamos los 10 jugadores con más partidas de cada color.

In [3]:
# Crear sesión de Spark
spark = SparkSession.builder \
    .appName("ContarJugadores") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

# Rutas
ruta_entrada = "/user/ajedrez/limpios/"  # Usa la carpeta de PGNs ya filtrados o limpios

# Leer archivos PGN desde HDFS
archivos_pgn = sc.wholeTextFiles(ruta_entrada + "*.pgn")  # (ruta, contenido)

def extraer_jugadores(ruta_y_contenido):
    _, contenido = ruta_y_contenido
    resultados = []

    pgn_stream = StringIO(contenido)
    while True:
        try:
            game = chess.pgn.read_game(pgn_stream)
            if game is None:
                break
            white = game.headers.get("White", "Unknown")
            black = game.headers.get("Black", "Unknown")
            resultados.append((white, "W"))
            resultados.append((black, "B"))
        except Exception:
            continue
    return resultados

# Extraer jugadores con colores
jugadores = archivos_pgn.flatMap(extraer_jugadores)

# Contar partidas por jugador y color
conteo = jugadores.map(lambda x: ((x[0], x[1]), 1)) \
                  .reduceByKey(lambda a, b: a + b)

# Reestructurar a formato ((jugador), {"W": n1, "B": n2})
conteo_por_jugador = conteo.map(lambda x: (x[0][0], {x[0][1]: x[1]})) \
                           .reduceByKey(lambda a, b: {
                               "W": a.get("W", 0) + b.get("W", 0),
                               "B": a.get("B", 0) + b.get("B", 0)
                           })

# Obtener top 10 jugadores por blancas y negras
top_blancas = conteo_por_jugador.sortBy(lambda x: x[1].get("W", 0), ascending=False).take(10)
top_negras = conteo_por_jugador.sortBy(lambda x: x[1].get("B", 0), ascending=False).take(10)

# Mostrar resultados
print("Top 10 jugadores con más partidas con blancas:")
for jugador, conteos in top_blancas:
    print(f"{jugador}: {conteos.get('W', 0)} partidas")

print("\n Top 10 jugadores con más partidas con negras:")
for jugador, conteos in top_negras:
    print(f"{jugador}: {conteos.get('B', 0)} partidas")

spark.stop()


🎯 Top 10 jugadores con más partidas con blancas:
?: 111227 partidas
Sarana,A: 173 partidas
Bluebaum,M: 154 partidas
Ambartsumova,K: 152 partidas
Rustemov,A: 150 partidas
Bortnyk,Olexandr: 149 partidas
Makarian,Rudik: 144 partidas
Zhigalko,S: 141 partidas
Martinez Alcantara,Jose Eduardo: 141 partidas
Barria Zuniga,D: 139 partidas

🎯 Top 10 jugadores con más partidas con negras:
?: 111227 partidas
Sarana,A: 167 partidas
Ambartsumova,K: 162 partidas
Rustemov,A: 151 partidas
Bluebaum,M: 148 partidas
Bortnyk,Olexandr: 147 partidas
Makarian,Rudik: 136 partidas
Vlassov,N: 133 partidas
Nakamura,Hi: 132 partidas
Martinez Alcantara,Jose Eduardo: 132 partidas


### (Opcional) Guardar partidas de una selección de jugadores

In [1]:
from pyspark.sql import SparkSession
from io import StringIO
import os
import chess.pgn
import subprocess

# Crear sesión Spark
spark = SparkSession.builder \
    .appName("GuardarPartidasJugadores") \
    .getOrCreate()
sc = spark.sparkContext

# Rutas en HDFS
ruta_entrada = "/user/ajedrez/limpios/"
ruta_salida = "/user/ajedrez/jugadores_objetivo/"
archivo_salida_local = "/tmp/partidas_jugadores.pgn"

# Crear carpeta de salida si no existe
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", ruta_salida])

# Lista de jugadores objetivo
jugadores_objetivo = ["Carlsen,M", "Sarana, A", "Duda, J"]

# Leer archivos PGN
archivos_pgn = sc.wholeTextFiles(ruta_entrada + "*.pgn")  # (ruta, contenido)

def filtrar_partidas_objetivo(ruta_y_contenido):
    _, contenido = ruta_y_contenido
    pgn_stream = StringIO(contenido)
    partidas = []

    while True:
        try:
            game = chess.pgn.read_game(pgn_stream)
            if game is None:
                break

            white = game.headers.get("White", "")
            black = game.headers.get("Black", "")

            if white in jugadores_objetivo or black in jugadores_objetivo:
                exporter = chess.pgn.StringExporter(headers=True, variations=True, comments=True)
                partidas.append(game.accept(exporter) + "\n\n")

        except Exception:
            continue

    return partidas

# Aplicar filtrado
partidas_filtradas = archivos_pgn.flatMap(filtrar_partidas_objetivo)

# Unir todas las partidas en un único archivo
partidas_texto = partidas_filtradas.collect()

# Guardar archivo temporalmente en el contenedor
with open(archivo_salida_local, "w", encoding="utf-8") as f:
    for partida in partidas_texto:
        f.write(partida)

# Subir a HDFS
ruta_archivo_hdfs = ruta_salida + "partidas_jugadores.pgn"
subprocess.run(["hdfs", "dfs", "-rm", ruta_archivo_hdfs], stderr=subprocess.DEVNULL)
subprocess.run(["hdfs", "dfs", "-put", archivo_salida_local, ruta_archivo_hdfs])

print(f"\n Se guardaron {len(partidas_texto)} partidas en: {ruta_archivo_hdfs}")

spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 21:39:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                


 Se guardaron 214 partidas en: /user/ajedrez/jugadores_objetivo/partidas_jugadores.pgn
